### Ingestion del archivo "genre.csv"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

#### Paso 1 - Leer el archivo CSV utilizando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, DateType

In [0]:
genre_schema = StructType(fields =[

    StructField("genreId", IntegerType(), False),
    StructField("genreName", StringType(), True)
])

In [0]:
genre_df = spark.read \
    .option("header",True) \
    .schema(genre_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/genre.csv")

#### Paso 3 - Cambiar el nombre de las columnas segun lo "requerido"

In [0]:
genre_renamed_df = genre_df \
    .withColumnRenamed("genreId","genre_id") \
    .withColumnRenamed("genreName","genre_name")

#### Paso 3 - Agregar una columna "ingestion date" al dataframe

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import lit

In [0]:
genre_final_df = add_ingestion_date(genre_renamed_df) \
.withColumn("enviroment", lit(dbutils.widgets.get("p_enviroment"))) \
.withColumn("file_date", lit(v_file_date))


#### Paso 4 - Escribir datos en Datalake en formato parquet

In [0]:
#genre_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/genre")
genre_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.genre");